In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-11 22:25:36--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.46MB/s    in 0.2s    

2022-01-11 22:25:37 (5.46 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
toys_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True)
toys_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [ ]:
toys_df.count()

4864249

In [ ]:
from pyspark.sql.functions import to_date
from pyspark.sql.types import IntegerType
review_id_df = toys_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = review_id_df.withColumn('customer_id',review_id_df.customer_id.cast(IntegerType()))
review_id_df = review_id_df.withColumn('product_parent',review_id_df.product_parent.cast(IntegerType()))
review_id_df = review_id_df.dropDuplicates(['review_id'])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10002QPRRUS7P|   14423478|B00911ZGUI|     971813725| 2012-11-17|
|R10005KUQC408N|   31924705|B004RSGBOA|     576484403| 2015-04-05|
|R10009VEIIYI8Y|   43392281|B0090HDQLE|      50301231| 2013-02-15|
| R1000HIXH8AKD|   48513712|B00E4LFI7Q|     633791254| 2014-05-26|
|R1000PYTO6LKGT|   46353316|B000BQVCD2|     853656505| 2009-12-30|
|R1000QFBCM4Y5O|   32054389|B00IMHTO7S|     290855054| 2015-01-06|
|R1000QU6Q8QYNH|   35699911|B000GKE2N0|     736102497| 2013-09-07|
|R1000T0HK1XXCA|   36339980|B005BGP4HC|     844923801| 2013-01-16|
|R1000Y4CDNNA3F|   27322612|B003AU9AGM|     547949031| 2013-11-05|
|R10010UI9B3O7L|   10206159|B00BQYQY8Q|     457646089| 2014-11-26|
|R10011CUY604R8|   26510939|B004478GI8|     775435608| 2012-03-15|
|R10019WINJX8XK|    8243624|B00Z4B94DE|     334758693| 2015-08

In [ ]:
products_df = toys_df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00IANTUPU|Lego, Star Wars, ...|
|B002TMK962|Rock - Concert Ti...|
|B00AOTTSQY|Adult & Children ...|
|B00CH3O5S6|48 Wholesale Wood...|
|B00MF3SAF2|Teenitor 3.7V 680...|
|B004D0C5WY|ThinkMax Silver B...|
|B00NHQHJZS|LEGO Technic Arct...|
|B00NQJ2DBG|Super Bright Long...|
|B0082IGA06|Yu-Gi-Oh! - Dark ...|
|1609581520|American Girl Car...|
|B004RGYVLC|DC Comics POP Her...|
|B0053EX3D4|Inuyasha Kirara T...|
|B00OTLHNI8|Bloco Toys T-Rex ...|
|B00BXEYI8W|Kid Galaxy Robot ...|
|B00JUG2NSM|Weiss Schwarz Kil...|
|B00LVPY252|Maxx Action 30" T...|
|B00IGRHWSC|Mattel Hot Wheels...|
|B00U6FU3YM|Educational Insig...|
|B00Z7FS3Q6|Beach Ball Set, 3...|
|B000WEAGTY|Inflatable Tradit...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
reviews_df = toys_df.select(["review_id", "review_headline", "review_body"])
reviews_df.show()

+--------------+--------------------+--------------------+
|     review_id|     review_headline|         review_body|
+--------------+--------------------+--------------------+
| RDIJS7QYB6XNR|          Five Stars|        Excellent!!!|
|R36ED1U38IELG8|Good quality trac...|Great quality woo...|
| R1UE3RPRGCOLD|           Two Stars|Cards are not as ...|
|R298788GS6I901|my daughter loved...|my daughter loved...|
|  RNX4EXOBBPN5|     DONT BUY THESE!|Do not buy these!...|
|R3BPETL222LMIM|          Five Stars|Great item. Pictu...|
|R3SORMPJZO3F2J|         Three Stars|To keep together,...|
|R2RDOJQ0WBZCF6|          Five Stars|I was pleased wit...|
|R2B8VBEPB4YEZ7|          Five Stars|    Children like it|
|R1CB783I7B0U52|Shame on the sell...|Showed up not how...|
| R2D90RQQ3V8LH|          Five Stars|Really liked thes...|
|R1Y4ZOUGFMJ327|   Nice huge balloon|Nice huge balloon...|
|R2BUV9QJI2A00X|          Five Stars|          Great deal|
| RSUHRJFJIRB3Z|          Four Stars|       As Advertise

In [ ]:
customers_df = toys_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   49997672|             2|
|   27435626|             2|
|   50312871|            24|
|   43801871|             2|
|   33212984|             2|
|   41500251|             4|
|   11234247|             2|
|   36739867|             8|
|   31341684|            65|
|    2668626|             1|
|   31325270|             5|
|   15293494|             1|
|    7549374|             1|
|   24268456|             2|
|    2529063|             1|
|   48244313|             2|
|     255431|             1|
|   50583551|            17|
|   24453085|             1|
|   21596603|             2|
+-----------+--------------+
only showing top 20 rows



In [22]:
vine_df = toys_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df = vine_df.withColumn('star_rating',vine_df.star_rating.cast(IntegerType()))
vine_df = vine_df.withColumn('helpful_votes',vine_df.helpful_votes.cast(IntegerType()))
vine_df = vine_df.withColumn('total_votes',vine_df.total_votes.cast(IntegerType()))
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RDIJS7QYB6XNR|          5|            0|          0|   N|
|R36ED1U38IELG8|          5|            0|          0|   N|
| R1UE3RPRGCOLD|          2|            1|          1|   N|
|R298788GS6I901|          5|            0|          0|   N|
|  RNX4EXOBBPN5|          1|            1|          1|   N|
|R3BPETL222LMIM|          5|            0|          0|   N|
|R3SORMPJZO3F2J|          3|            2|          2|   N|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|
|R1CB783I7B0U52|          1|            0|          1|   N|
| R2D90RQQ3V8LH|          5|            0|          0|   N|
|R1Y4ZOUGFMJ327|          5|            0|          0|   N|
|R2BUV9QJI2A00X|          5|            0|          1|   N|
| RSUHRJFJIRB3Z|          4|            

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://mypostgresdb.cfkxyybjzccp.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root",
          "password": "markdavid",
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id_df \
    .groupby(['review_id']) \
    .count() \
    .where('count > 1') \
    .sort('count', ascending=False) \
    .show()

+---------+-----+
|review_id|count|
+---------+-----+
+---------+-----+



In [ ]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)